# Project Title
## Data Engineering Capstone Project

### Project Summary
The purpose of this project is to create an ETL pipeline that wrangles data on immigration, demographics and environmental factors (specifically temperature), in order to be able to glean insights about what factors may make certain US destination popular for international visits. Example questions that business users may want answered include:
* Is there a correlation between a visitor's home temperature and the temperature of their chosen destination?
* Do people from certain countries prefer destination cities where certain ethnicities are better represented?
* Are particular destinations more popular with holders of visas of a certain type?

We'll be using Spark to process the data.

The project is broken down into 5 steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Clean Up the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Let's get ready by importing the libraries and tools we'll be using...

import pandas as pd
import helpers
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, upper, first, dayofmonth, weekofyear, month, year, dayofweek, udf, monotonically_increasing_id, datediff
from pyspark.sql.types import StringType

# ...and initializing our SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

## Step 1: Scoping the Project and Gathering Data

### Scope 
We have various data on immigration, travel, temperature and demographics, from various sources. The business need is to be able to query the data to see what factors may affect immigration or US travel destination choices. More specifically, the goal is to create a set of relational tables against which we can run queries to look for patterns between chosen immigration or travel destination, and demographic and environmental factors.

To accomplish this, Spark will be used, as it is well suited to process large amounts of data across multiple servers. If we wanted to take the project further, we could migrate the output data into a suitable RDBMS, like Redshift, and we could automate the pipleline with Airflow; this, however, is outside the scope of this particular project. The primary goal and scope of the task here is to explore the data, model it and load it into relational tables suitable for analytic querying.


### Describing and Gathering the Data 
The source data are the following:

* _I94 Immigration Data_: This data comes from the [US National Tourism and Trade Office](https://www.trade.gov/i-94-arrivals-program). It consists of the 2016 I-94 visitor arrivals data, providing information on arrivals for US visitors who stay 1 night or more. The link to the original dataset is defunct, but a copy of the data was captured and provided by Udacity.
* _World Temperature Data_: [This dataset](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data) was sourced from Kaggle. It consists of a table of global land temperatures by city and by month, from approximately 1750.
* _U.S. City Demographic Data_: [This data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) comes from OpenSoft. The data is compiled from the US Census Bureau's 2015 American Community Survey. It contains demographic information for all US cities that have a population of 65,000 or more. 
* _Airport Code Table_: [This](https://datahub.io/core/airport-codes#data) is a table of airport codes and corresponding cities.

To create the relational tables that will allows us to explore demographic and environmental correlations, we will only need data from the first three sets. We will not be using the _Airport Code Table_ dataset.

## Step 2: Exploring and Cleaning Up the Data
Let's identify data quality issues, like missing values, duplicate data, etc.

Some of this data was explored "offline", so to speak; information was gathered by visiting the original sources for the datasets or via supplemental contextual information. The i94 dataset, particularly, benefited from this; the definitions of the columns was understood primarily through looking at the `I94_SAS_Labels_Descriptions.SAS` file located in the same directory as this README file.

### Exploration
#### Immigration Data

In [2]:
# Read in the i94 data
i94_df = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [ ]:
# Get a preview
i94_df.limit(5).toPandas()

In [ ]:
# Show the total of null or NaN values per column
helpers.show_total_missing_values(i94_df)

#### Demographic Data

In [3]:
# Read in the demographic data
demog_df = spark.read.csv('us-cities-demographics.csv', inferSchema=True, header=True, sep=';')

In [ ]:
# Get a preview
demog_df.limit(5).toPandas()

In [ ]:
# Show the total of null or NaN values per column
helpers.show_total_missing_values(demog_df)

#### Temperature Data

In [4]:
# Read in the temperature data
temp_df = spark.read.csv('../../data2/GlobalLandTemperaturesByCity.csv', header=True, inferSchema=True)

In [ ]:
# Get a preview
temp_df.limit(5).toPandas()

In [ ]:
# Show the total of null or NaN values per column, but first we cast the datetime column to string
temp_df_string = temp_df.withColumn("dt",col("dt").cast(StringType()))
helpers.show_total_missing_values(temp_df_string)

### Clean Up

#### Immigration Data
If we look again at the table of total missing values per column in the _Exploration_ section, we can see that three columns have a total of more than 85% missing values: `occup`, `entdepu`, and `insnum`. These can be dropped.

Taking this a step further, as we'll see shortly in our data model, there are only 11 columns we're interested in keeping. Let's select these to make the dataframe easier to work with and more legible to display. While we're at it, let's rename those columns and drop duplicates.

In [5]:
i94_df = i94_df.select(col('cicid').alias('id'),
                             col('i94cit').alias('citizenship_country'),
                             col('i94res').alias('residence_country'),
                             col('i94port').alias('port_of_entry'),
                             col('i94addr').alias('destination_state'),
                             col('arrdate').alias('arrival_date'),
                             col('depdate').alias('departure_date'),
                             col('i94bir').alias('age'),
                             col('i94visa').alias('visa_type'),
                             col('dtaddto').alias('admitted_until'),
                             col('gender').alias('gender')
                            ).dropDuplicates().cache()

We'll also convert the SAS dates in `arrdate` and `depdate`, and the string date in `dtaddto`, to PySpark dates.

In [6]:
i94_df = i94_df.withColumn('arrival_date', helpers.convert_sas_date_to_datetime(i94_df.arrival_date)).cache()
i94_df = i94_df.withColumn('departure_date', helpers.convert_sas_date_to_datetime(i94_df.departure_date)).cache()
i94_df = i94_df.withColumn('admitted_until', to_date(col('admitted_until'),'MMddyyyy')).cache()

In [ ]:
i94_df.limit(5).toPandas()

We can see that the fields `citizenship_country`, `residence_country`, and `state_of_arrival` are referenced by codes. These need to be cross-referenced with a list of codes that is currently only available in .SAS file. We'll update these values with their full names.

We also have the `port_of_entry` field in the i94 table, which we can decompose into port city, port state, and port country. Doing this will later on allow us to add the port code to the temperature and demographic tables, which will allow us to keep our fact table simple, as a single field (`port_of_entry`) will be able to link three dimension tables together. 

All of this will serve to:
1. make the values in our tables more meaningful, and
2. make it easier to join tables when querying by reducing the number of joins and tables involved.

In [7]:
# Read our i94 fields and values legend 
with open('I94_SAS_Labels_Descriptions.SAS') as f:
    i94_desc = f.readlines()

In [8]:
# get country names by code
country_codes = {}
for countries in i94_desc[10:298]:
    pair = countries.split('=')
    code, country = pair[0].strip(), pair[1].strip().strip("'")
    country_codes[code] = country

In [9]:
origin_countries_df = spark.createDataFrame(list(country_codes.items()), ['code', 'country'])

In [ ]:
origin_countries_df.show(5)

In [10]:
# get city names and state abbreviations by numerical codes
port_cities = {}
port_states = {}
broken_fields = {}
for cities in i94_desc[303:893]:   
    pair = cities.split('=')
    code, location = pair[0].strip("\t").strip().strip("'"), pair[1].strip('\t').strip()
    city_and_state = location.split(',')
    if len(city_and_state) == 2:
        city, state = city_and_state[0].strip().strip("'"), city_and_state[1].strip().strip("'").strip()
        port_cities[code] = city
        port_states[code] = state

In [11]:
port_cities_df = spark.createDataFrame(list(port_cities.items()), ['code', 'city'])

In [ ]:
port_cities_df.show(5)

In [12]:
# get state names by state abbreviation
state_abbrs = {}
for states in i94_desc[982:1035]:   
    pair = states.split('=')
    abbr, state = pair[0].strip("\t").strip().strip("'"), pair[1].strip('\t').strip().strip("'")
    if 'N.' in state:
        state = state.replace('N.','NORTH')
    if 'S.' in state:
        state = state.replace('S.','SOUTH')
    if 'W.' in state:
        state = state.replace('W.','WEST')
    if 'DIST.' in state:
        state = state.replace('DIST.','DISTRICT')
    state_abbrs[abbr] = state

In [13]:
port_countries = {}
for code in port_states: 
    if port_states[code] in state_abbrs:
        port_states[code] = state_abbrs[port_states[code]]
        port_countries[code] = 'UNITED STATES'
    else:
        port_countries[code] = port_states[code]
        port_states[code] = None

In [14]:
state_abbrs_df = spark.createDataFrame(list(state_abbrs.items()), ['abbr','state'])
port_states_df = spark.createDataFrame(list(port_states.items()), ['code','state'])
port_countries_df = spark.createDataFrame(list(port_countries.items()), ['code','country'])

In [ ]:
state_abbrs_df.show(5)

In [ ]:
port_states_df.show(5)

In [ ]:
port_countries_df.show(5)

In [15]:
# create views from the dictionary dataframes we just created, 
# so we can use sql to add meaningful port city, state and country values to our i94 table
i94_df.createOrReplaceTempView('i94_view')
origin_countries_df.createOrReplaceTempView('origin_countries_view')
state_abbrs_df.createOrReplaceTempView('state_abbrs_view')
port_cities_df.createOrReplaceTempView('port_cities_view')
port_states_df.createOrReplaceTempView('port_states_view')
port_countries_df.createOrReplaceTempView('port_countries_view')

In [16]:
# clean up our i94 table by converting codes to human-readable values
i94_clean_df = spark.sql("""
    SELECT 
        i94.id AS immigration_id,
        i94.age AS age,
        i94.gender AS gender,
        c_cit.country AS citizenship_country,
        c_res.country AS residence_country,
        i94.arrival_date AS arrival_date,
        i94.departure_date AS departure_date,
        i94.visa_type AS visa_type,
        i94.admitted_until AS admitted_until,
        i94.port_of_entry AS port_of_entry,
        p_cit.city AS port_city,
        ps.state AS port_state,
        p_country.country AS port_country,
        sa.state AS destination_state
    FROM i94_view i94
    
    -- retrieve the country names of the code values in i94.citizenship_country
    LEFT JOIN origin_countries_view c_cit
    ON i94.citizenship_country = c_cit.code
    
    -- retrieve the country names of the code values in i94.residence_country
    LEFT JOIN origin_countries_view c_res
    ON i94.residence_country = c_res.code
    
    -- retrieve the city names of the code values in i94.port_of_entry
    LEFT JOIN port_cities_view p_cit
    ON i94.port_of_entry = p_cit.code
    
    -- retrieve the state names of the code values in i94.port_of_entry, if applicable
    LEFT JOIN port_states_view ps
    ON i94.port_of_entry = ps.code
    
    -- retrieve the country names of the code values in i94.port_of_entry
    LEFT JOIN port_countries_view p_country
    ON i94.port_of_entry = p_country.code
    
    -- retrieve the full state names of the state abbreviations in i94.destination_state
    LEFT JOIN state_abbrs_view sa
    ON i94.destination_state = sa.abbr
""")

In [ ]:
i94_clean_df.show(5)

In [ ]:
# check the distribution of our destination_state values. Make sure we didn't accidentally end up with all nulls (for example)
i94_clean_df.groupBy('destination_state').count().show(100)

In [ ]:
# check the distribution of port_state values. Make sure we didn't accidentally end up with all nulls (for example)
i94_clean_df.groupBy('port_city').count().show(100)

#### Demographic Data
We'll start by renaming the columns and uppercasing the city and state names, to have consistent data formatting across our different datasets.

In [17]:
demog_df = demog_df.select(upper(col('City')).alias('city'),
                             upper(col('State')).alias('state'),
                             col('Median Age').alias('median_age'),
                             col('Male Population').alias('male_population'),
                             col('Female Population').alias('female_population'),
                             col('Total Population').alias('total_population'),
                             col('Number of Veterans').alias('total_veterans'),
                             col('Foreign-born').alias('foreign_born'),
                             col('Average Household Size').alias('average_household_size'),
                             col('Race').alias('race'),
                             col('Count').alias('race_count')
                            ).dropDuplicates().cache()

In [ ]:
demog_df.show(5)

While we may want denormalized data in our fact and dimension tables, we want to make sure that the denormalizations make sense for our purposes. If we look closely, we can observe that the `Race` column in the demographic data has a small number of unnecessarily repeating values. Barring a few anomalies, each cityappears as five separate rows with the same data for every field, except for the `Race` column, and the `Count`, which refers to the total number of residents who identify with the associated race in a given row. 

To simplify our table and our querying, we can simply pivot the `Race` column, meaning that we would take its five possible values and make each of them a single column. The values in the `Count` column would become the values for the associated `Race` column.

In [18]:
# Turn the Race column into five distinct race columns
demog_df = demog_df.groupBy("city",
                            "state",
                            "median_age",
                            "male_population",
                            "female_population",
                            "total_population",
                            "total_veterans", 
                            "foreign_born", 
                            "average_household_size"
                           ).pivot("race").agg(first("race_count"))

In [ ]:
demog_df.show(5)

In [19]:
# Tidy the ethnicity column names
demog_df = demog_df.select(col('city'),
                           col('state'),
                           col('median_age'),
                           col('male_population'),
                           col('female_population'),
                           col('total_population'),
                           col('total_veterans'),
                           col('foreign_born'),
                           col('average_household_size').alias('avg_household_size'),
                           col('American Indian and Alaska Native').alias('indigenous'),
                           col('Asian').alias('asian'),
                           col('Black or African-American').alias('black'),
                           col('Hispanic or Latino').alias('latinx'),
                           col('White').alias('white'),
                          ).dropDuplicates().cache()

In [ ]:
demog_df.count()

In [ ]:
demog_df.limit(5).toPandas()

#### Temperature Data

We saw in our data exploration that there were rows with missing AverageTemperature. These rows will be useless for our purposes, so we will drop them.

In [20]:
temp_df=temp_df.filter(temp_df.AverageTemperature != 'NaN')

We will also see shortly in our data model that the columns `AverageTemperatureUncertainty`, `Latitude` and `Longitude` are not relevant to our inquiry, so we will drop them and take the opportunity to rename our remaining columns, as well as to upprecase the city and country values.

In [21]:
temp_df = temp_df.select(col('dt').alias('date'),
                             col('AverageTemperature').alias('avg_temperature'),
                             upper(col('City')).alias('city'),
                             upper(col('Country')).alias('country')
                            ).dropDuplicates().cache()

In [ ]:
temp_df.limit(5).toPandas()

Additionally, our i94 dataset is specific to the year 2016. Our inquiry concerns itself with the factors that may have influenced visitors to choose the destinations they chose. The temperature dataset contains data from as far back as 1743. For our context, we will assume that people would not concern themselves with average temperatures more than 10 years old when choosing where to go in the short term. Hence, we will drop historical data from before 2006.

In [22]:
# Drop pre-2006 data, and make sure the date field doesn't get a midnight timestamp added (useless for our purposes)
temp_df = temp_df.filter(temp_df.date >= '2006-01-01').withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
# temp_df.count()

In [ ]:
temp_df.show(5)

## Step 3: Define the Data Model
### 3.1 Conceptual Data Model
Data will be transferred into a star schema (one fact table where each dimension tables is referenced by a row; additional, derived information can also be added to the fact table). This schema design facilitates analytical querying, as it denormalizes data according to the particular aspects of the data we want to dig further into. In other words, it's optimized to minimize the number of JOINs required to query the data. 

#### Fact Table

**visits_fact** - immigration events
- visit_id, immigration_id, port_code, arrival_date, length_of_stay

#### Dimension Tables

**immigrations_dim** - i94 immigration details
- immigration_id, age, gender, citizenship_country, residence_country, arrival_date, departure_date, visa_type, admitted_until, port_of_entry, port_city, port_state, port_country, destination_state

**demographics_dim** - demographic data per major US city
- city_id, port_code, city, state, median_age, male_population, female_population, total_population, total_veterans, foreign_born, avg_household_size, indigenous, asian, black, latinx, white

**temperature_dim** - average temperatures per month from 2006-2013 for global cities
- temperature_id, port_code, date, avg_temperature, city, country

**date_dim** - dates of arrivals for visits
- date, day, week, month, year, weekday

![data model](./pipeline_star_schema.jpg "data model")

### 3.2 Mapping Out Data Pipelines
The steps necessary to pipeline the data into the above data model will be as follows:
1. Clean the i94, temperature and demographics datasets (already performed)
2. Create a date_dim dimension table based on the arrival_date values in the immigrations table, and extract the desired fields
3. Create temperature_dim dimension table: 
    1. Load the existing, cleaned up temperature table, adding a unique id column
    2. Add a port_code field based on the city value
4. Create a demographic_dim dimension table:
    1. Load the existing, cleaned up demographics table, adding a unique id column
    2. Add a port_code field based on the city value
5. Create the immigrations_dim table by loading the i94 table (we did a thorough job cleaning it up, knowing ahead of time what our data model was going to be; it already looks how we want it to)
6. Create the visits_fact table with a derived length_of_stay column, mapping immigration_id from immigrations_dim, a port_code that can be referenced in temperature_dim and demographics_dim, and the arrival_date associated with the immigration_id
7. Write these tables to parquet files that can be queried for analysis

## Step 4: Run Pipelines to Model the Data 
### 4.1 Create the data model
Build the data pipelines to create the data model.

In [23]:
# Create date_dim table
date_dim_df = i94_clean_df.withColumn("date", col("arrival_date"))\
                .withColumn("day", dayofmonth("arrival_date"))\
                .withColumn("week", weekofyear("arrival_date"))\
                .withColumn("month", month("arrival_date"))\
                .withColumn("year", year("arrival_date"))\
                .withColumn("weekday", dayofweek("arrival_date"))
                        
date_dim_df = date_dim_df.select("date", "day", "week", "month", "year", "weekday").drop_duplicates()

In [33]:
# UDF to add a port_code column to our temperature and demographics tables
@udf()
def get_city_port(city):
    for key in port_cities:
        if city in port_cities[key]:
            return key

In [36]:
# Create temperature_dim table
temperature_dim_df = temp_df.select(monotonically_increasing_id().alias("temperature_id"),
                                    col("date"),
                                    col("avg_temperature"),
                                    col("city"),
                                    col("country"),
                                   ).withColumn("port_code", get_city_port(temp_df.city))

In [42]:
# Create demographics_dim table
demographics_dim_df = demog_df.select(monotonically_increasing_id().alias("city_id"),
                                    col("city"),
                                    col("state"),
                                    col("median_age"),
                                    col("male_population"),
                                    col("female_population"),
                                    col("total_population"),
                                    col("total_veterans"),
                                    col("foreign_born"),
                                    col("avg_household_size"),
                                    col("indigenous"),
                                    col("asian"),
                                    col("black"),
                                    col("latinx"),
                                    col("white"),
                                   ).withColumn("port_code", get_city_port(demog_df.city))

In [44]:
# Create immigrations_dim table
immigrations_dim_df = i94_clean_df

In [51]:
# Create visits_fact table
from pyspark.sql.functions import datediff

visits_fact_df = immigrations_dim_df.select(monotonically_increasing_id().alias("visit_id"),
                                    col("immigration_id"),
                                    col("port_of_entry").alias("port_code"),
                                    col("arrival_date"),
                                    datediff(col("departure_date"),col("arrival_date")).alias("length_of_stay")
                                   )

In [53]:
# Create parquet files for analysis
%rm -rf ./results/
immigrations_dim_df.write.mode('append').partitionBy('age').parquet('./results/immigrations_dim.parquet')
demographics_dim_df.write.mode('append').partitionBy('state').parquet('./results/demographics_dim.parquet')
temperature_dim_df.write.mode('append').partitionBy('country').parquet('./results/temperature_dim.parquet')
date_dim_df.write.mode('append').parquet('./results/date_dim.parquet')
visits_fact_df.write.mode('append').partitionBy('arrival_date').parquet('./results/visits_fact.parquet')

### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

## Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.